In [ ]:
pip install transformers

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import nltk
from scipy.special import softmax

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer



genre = ["Comedy", "Drama" , "Action", "Romantic", "Horror", "Crime"]

MODEL = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=len(genre))


df = load_dataset("jahjinx/IMDb_movie_reviews")


mlb = MultiLabelBinarizer(classes=genre)

# Apply the encoding to the 'genres' column
def encode_labels(examples):
    examples['labels'] = mlb.fit_transform(examples['genres'])
    return examples

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = df.map(tokenize_function, batched=True)


#example_text = df['train'][225]['text']
#encoded_text = tokenizer(example_text, return_tensors='pt')
#output = model(**encoded_text)
#scores = output[0][0].detach().numpy()
#scores = softmax(scores)
#scores_dict = {
 #   'roberta_neg': scores[0],
  #  'roberta_pos': scores[1]
#}



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
import os
from transformers import TrainingArguments
from transformers import Trainer


os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(output_dir="test_trainer", per_device_train_batch_size=8)

trainer = Trainer(
    model = model, args=training_args, train_dataset=tokenized_datasets["train"]
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.462500
1000,0.355300
1500,0.339700
2000,0.331400
2500,0.289500
3000,0.314900
3500,0.286400
4000,0.288200
4500,0.287200
5000,0.202800


Step,Training Loss
500,0.462500
1000,0.355300
1500,0.339700
2000,0.331400
2500,0.289500
3000,0.314900
3500,0.286400
4000,0.288200
4500,0.287200
5000,0.202800


TrainOutput(global_step=13500, training_loss=0.2106685081764504, metrics={'train_runtime': 4807.7543, 'train_samples_per_second': 22.464, 'train_steps_per_second': 2.808, 'total_flos': 2.8417014521856e+16, 'train_loss': 0.2106685081764504, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model_path = "/content/drive/MyDrive/ML_Projects/saved_roberta_model"

# Save model and tokenizer
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/content/drive/MyDrive/ML_Projects/saved_roberta_model/tokenizer_config.json',
 '/content/drive/MyDrive/ML_Projects/saved_roberta_model/special_tokens_map.json',
 '/content/drive/MyDrive/ML_Projects/saved_roberta_model/vocab.json',
 '/content/drive/MyDrive/ML_Projects/saved_roberta_model/merges.txt',
 '/content/drive/MyDrive/ML_Projects/saved_roberta_model/added_tokens.json',
 '/content/drive/MyDrive/ML_Projects/saved_roberta_model/tokenizer.json')

In [ ]:
print(trainer.evaluate())

ValueError: Trainer: evaluation requires an eval_dataset.